### Model Selection 학습

학습/테스트 데이터 : train_test_split()

model select 모듈 - 학습 데이터와 텍스트 데이터 세트를 분리 또는 교차 검증분할 및 평가, 하이퍼 파라미터 튜닝을 위한 다양한 함수 제공

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


**model selection 로직 비적용한 논리적 모델링 오류**

In [4]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris.data  # 독립변수, feature, 
train_label = iris.target # 종속변수, label, class, target

# 학습
dt_clf.fit(train_data, train_label)

# 예측 해 주세요
pred = dt_clf.predict(train_data)

# 예측된 결과가 100% 일치되었다란 의미 
print('예측 정확도: ', accuracy_score(train_label, pred) )

예측 정확도:  1.0


<hr>

**1차 개선 : train_test_split()로 학습, 테스트 데이터 분할**

In [5]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()

iris_data = iris.data  # 독립변수, feature, 
iris_label = iris.target # 종속변수, label, class, target

X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_label, test_size=0.4, random_state=121)
dt_clf = DecisionTreeClassifier(random_state=11)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도 : {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도 : 0.9667


In [6]:
train_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

<hr>

**2차 개선 : 교차 검증**

In [19]:
from sklearn.model_selection import StratifiedKFold as kfold
import numpy as np

In [20]:
iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier()

# 5개의 폴드 셋트로 분할
kfold = KFold(n_splits=5)

# 평가한 결과치를 적재하게 되는 리스트
cv_accuracy = []

In [21]:
n_iter = 0

# 폴드별 학습용, 검증용 row index를 array 반환
for train_index, test_index in kfold.split(features):
    # 학습용, 검증용 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    
    # 반복시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    cv_accuracy.append(accuracy)
print(cv_accuracy)
print(np.mean(cv_accuracy))    
cv_accuracy = []

[1.0, 0.9667, 0.8667, 0.9333, 0.7667]
0.90668


**학습 / 검증 레이블 데이터 값의 분포도 확인**

분포도 풀균형 확인 <br>
학습 데이터로 사용되지 않은 이상 예측이 부정확<br>
해결책 : StrtifiedKFold API 활용

<hr>
**Stratified K 폴드**

- 불균형한(imbalanced) 분포도를 가진 레이블(결정 class) 데이터 집합을 위한 K 폴드 방식
- 불균형한 분포도란? 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것 의미
- K 폴드가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배하지 못하는 경우의 문제 해결
- 원리 : 원본 데이터의 레이블 분포를 먼저 고려 -> 이 분포와 동일하게 학습과 검증 데이터 세트를 분배